# Actividad 02 - Índices y Algoritmos

En esta actividad vamos a entender en más detalle cómo funciona la interacción de un sistema de bases de datos con el disco duro. Dispones de: 

- Una módulo llamado `disk_sim.py` que dispone de ciertas funciones que simulan la interacción con las páginas del disco duro.
- Una clase llamada `ArtworkTable(id_artwork, name_artwork)`, que simula el comportamiento a bajo nivel de una tabla con un campo `int` y un `string`. En este caso la instancia es sobre **Obras de arte** y sus campos corresponde a un identificador y al nombre de la obra.
- 3 archivos que corresponden a "páginas" del disco duro. Estas páginas son de 6 líneas: (1) las 5 primeras representan casilleros donde encontraras tuplas (escritas en hexadecimal) o tienen un flag `<EMPTY>` en el caso de estar disponibles para recibir una tupla, y (2) una línea que indica cuál es el nombre del archivo que representa la siguiente página.

**Importante**: En esta actividad **no tienes que modificar en nada** el archivo `disk_sim.py`, además debes tener cuidado de no corromper los archivos que representan páginas de disco. Si crees que los corrompiste, deberías volver a los archivos originales. Esto, como se muestra más adelante, lo haces con la función `create()` del módulo `crate_data`.

## Acerca del modulo `disk_sim.py`

Lo primero que vamos a hacer es importar las funciones del el módulo `disk_sim.py`:

In [2]:
from disk_sim import (create_page, insert_into_pos,
                      get_element_from_page, delete_from_pos, 
                      read_hexa_tuple, read_next_page, 
                      set_next_page, tuple_to_hexa_string)

A lo largo de tu tarea no necesitarás conocer lo que hacen estas funciones, pues no las vas a usar directamente. De todas formas vamos a explicar lo que hace cada una:

- `create_page`: crea una nueva página nueva con 5 casilleros vacíos y un puntero a la siguiente página vacío.
- `insert_into_pos`: inserta un elemento en una determinada posición. El casillero debe estar vacío.
- `get_element_from_page`: obtiene el elemento de una determinada posición.
- `delete_from_pos`: borra un elemento de una determinada posición, seteando el casillero como `<EMPTY>`.
- `read_hexa_tuple`: método que transforma una tupla escrita en hexadecimal a una tupla de Python.
- `read_next_page`: entrega el nombre de la siguiente página.
- `set_next_page`: define el nombre de la página siguiente. Si ya existe una puntero a la siguiente página se sobrescribe.

Para más detalles sobre los argumentos de cada función puedes abrir el módulo y leer los comentarios.

## Parte 1 (4 pts): consultas sobre una tabla sin índices

La clase `ArtworkTable` sirve para traer las páginas que le corresponden a memoria. Recordemos que todas las páginas tienen 5 bloques, cada uno puede o no estar ocupado con una tupla. Lo primero que haremos será cargar esta clase. Cada uno de los métodos tiene documentado lo que hace, así que recomiendo que prestes atención a esto.

In [3]:
class ArtworkTable:
    def __init__(self, initial_page, size=5):
        self.root_page = initial_page
        self.size = size
        self.before_first()
        
    def before_first(self):
        '''
        Método que inicia la tabla. Setea el puntero al bloque actual como -1 y la página actual como la raiz.
        Luego se carga el contenido de la página inicial como una lista de tuplas en la lista current_page_values.
        Esto es equivalente a setear el puntero que recorre los bloques a justo antes del primer casillero.
        '''
        self.current_block = -1
        self.current_page_name = self.root_page
        self.current_page_values = []
        self.init_page_values()
        
    def init_page_values(self):
        '''
        Método que lee la página actual a memoria. 
        Carga las tuplas en current_page_name y el nombre de la página siguiente.
        '''
        for i in range(self.size):
            pre_row = get_element_from_page(self.current_page_name, i)
            if pre_row != None:
                row = (int(pre_row[0]), pre_row[1])
            else:
                row = None
            self.current_page_values.append(row)
            
        self.next_page_name = read_next_page(self.current_page_name)
            
    def has_next_block(self):
        '''
        Método que retorna True si hay un siguiente bloque después de la ubicación del puntero.
        El bloque puede ser de esta página o de otra más.
        '''
        if self.current_block < len(self.current_page_values) - 1:
            return True
        elif self.next_page_name:
            return True
        return False
    
    def get_next_value_block(self):
        '''
        Método que aumenta en uno el puntero del bloque actual. 
        Si ya no quedan más bloques pasa a la página siguiente y deja el puntero en el casillero 0.
        '''
        if self.current_block < len(self.current_page_values) - 1:
            self.current_block += 1
            return self.current_page_values[self.current_block]
        else:
            self.current_block = 0
            self.current_page_name = self.next_page_name
            self.current_page_values = []
            
            self.init_page_values()
            
            return self.current_page_values[self.current_block]
        
    def create_and_set_next_page(self, pname):
        '''
        Método que, en caso de que no haya una página siguiente a la actual,
        crea una y la deja como la siguiente. Se debe ingresar el nombre de la siguiente página.
        '''
        if self.next_page_name != None:
            print("Ya hay una siguiente página")
            return
        create_page(pname)
        self.next_page_name = pname
        set_next_page(self.current_page_name, pname)
        
    def set_value_on_page(self, id_artwork, name_artwork):
        '''
        Añade a la posición actual (el atributo current_block) la tupla con id id_artwork
        y nombre name_artwork.
        '''
        self.current_page_values[self.current_block] = (id_artwork, name_artwork)
        hexa_string = tuple_to_hexa_string((id_artwork, name_artwork))
        insert_into_pos(self.current_page_name, self.current_block, hexa_string)
        
    def set_empty_on_page(self):
        '''
        Borra la tupla de la posición actual.
        '''
        self.current_page_values[self.current_block] = None
        delete_from_pos(self.current_page_name, self.current_block)
        
    def has_next_tuple(self):
        '''
        Tienes que implementar esta función.
        '''
        pass
    
    def insert_into_table(self, id_artwork, name_artwork):
        '''
        Tienes que implementar esta función.
        '''
        pass
    
    def get_next_tuple(self):
        '''
        Tienes que implementar esta función.
        '''
        pass
    
    def get_all_elements_from_table(self):
        '''
        Tienes que implementar esta función.
        '''
        pass
    
    def get_tuple_by_id(self, id_artwork):
        '''
        Tienes que implementar esta función
        '''
        pass
    
    # Las siguientes dos funciones son para el bonus
    
    def del_tuple_by_id(self, id_artwork):
        '''
        Puedes implementar esta función para el bonus
        '''
        pass
        
    def compact(self):
        '''
        Puedes implementar esta función para el bonus
        '''
        pass


### ¿Cómo funciona esta clase?

Lo que hace el constructor de esta clase es recibir una página del disco, que representa la primera página de la tabla. La clase carga una página de la relación a la vez en memoria. El atributo `self.current_block` nos dice en qué posición estamos parados en la página actualmente abierta. **Ojo**: son 5 casilleros y las posiciones van del 0 al 4.

Ahora vamos a usar el constructor para abrir la primera página y vamos a imprimir los atributos importantes de la clase.

In [4]:
obras = ArtworkTable('o-1')

print(f'La posición actual es: {obras.current_block}')
print(f'La página actual es: {obras.current_page_name}')
print(f'La página siguiente es: {obras.next_page_name}')
print('Los valores en esta página son:')
print(obras.current_page_values)

La posición actual es: -1
La página actual es: o-1
La página siguiente es: o-2
Los valores en esta página son:
[(2, 'La mona lisa'), (10, 'El juicio final'), None, (11, 'El Moisés'), (9, 'La última cena')]


Como vemos, el puntero está en la posición -1, la tabla no está ordenada y hay una siguiente página llamada `o-2`. Ahora vamos a usar la función `has_next_block` que nos dice si hay un siguiente casillero en la tabla. Dado que estamos en el "casillero" antes del primero (-1), deberíamos recibir `True` por respuesta, porque hay 5 casilleros más.

In [5]:
obras.has_next_block()

True

Como sabemos que hay un bloque siguiente, se lo vamos a pedir a la tabla con la función `get_next_value_block()`

In [6]:
obras.get_next_value_block()

(2, 'La mona lisa')

Podemos repetir este procedimiento un par de veces hasta recorrer todos los elementos y pasar a la siguiente página:

In [7]:
print(obras.get_next_value_block())
print(obras.get_next_value_block())
print(obras.get_next_value_block())
print(obras.get_next_value_block())

(10, 'El juicio final')
None
(11, 'El Moisés')
(9, 'La última cena')


Lo que hicimos fue recorrer los primeros 5 casilleros. Notamos que hay un `None` que nos puestra el casillero disponible. Ahora volveremos a preguntar si hay un siguiente casillero:

In [8]:
obras.has_next_block()

True

Nos responde `True`, porque existe un siguiente casillero pero en la página siguiente, así que al leer el siguiente bloque:

In [9]:
obras.get_next_value_block()

(6, 'El David')

Hemos pasado a la posición 0 de la siguiente página. Veamos los atributos del objeto `obras`:

In [10]:
print(f'La posición actual es: {obras.current_block}')
print(f'La página actual es: {obras.current_page_name}')
print(f'La página siguiente es: {obras.next_page_name}')
print('Los valores en esta página son:')
print(obras.current_page_values)

La posición actual es: 0
La página actual es: o-2
La página siguiente es: o-3
Los valores en esta página son:
[(6, 'El David'), (4, 'El nacimiento de Venus'), None, None, (5, 'La consagración de Napoleón')]


Y podemos seguir así hasta llegar al final. Ahora vamos a posicionarnos de nuevo "justo antes del primer elemento" para recorrer todas las páginas de la tabla:

In [11]:
obras.before_first()

while(obras.has_next_block()):
    print(obras.get_next_value_block())

(2, 'La mona lisa')
(10, 'El juicio final')
None
(11, 'El Moisés')
(9, 'La última cena')
(6, 'El David')
(4, 'El nacimiento de Venus')
None
None
(5, 'La consagración de Napoleón')
(1, 'La anunciación')
(7, 'La piedad')
(3, 'La primavera')
(8, 'La fuente de los 4 ríos')
None


Ahora vamos a mostrar como insertar un elemento. Para esto volveremos al inicio y llegaremos a una posición vacía.

In [ ]:
obras.before_first()
obras.get_next_value_block()
obras.get_next_value_block()
obras.get_next_value_block()

Volvemos a revisar que tenemos aquí:

In [ ]:
print(f'La posición actual es: {obras.current_block}')
print(f'La página actual es: {obras.current_page_name}')
print(f'La página siguiente es: {obras.next_page_name}')
print('Los valores en esta página son:')
print(obras.current_page_values)

Vemos que la posición actual es la 2, que corresponde a un elemento vacío, así que vamos a insertar un valor con el método `set_value_on_page`.

In [ ]:
obras.set_value_on_page(15, 'Una obra de prueba')

Y ahora volvemos a recorrer todas las páginas para mostrar que la tupla está ahí:

In [ ]:
obras.before_first()
while(obras.has_next_block()):
    print(obras.get_next_value_block())

Todo bien hasta acá. Ahora volveremos los datos a los originales (aquí te vas a dar cuenta que no siempre se insertan en el mismo orden). Al crear de nuevo los datos debes volver a instanciar el objeto `obras`.

In [ ]:
from create_data import create

create()
obras = ArtworkTable('o-1')

Lo último que nos queda por repasar es el método `create_and_set_next_page(self, pname)`. Este método solamente funciona si estamos parados en una página que no posea una página siguiente. Esto es que el atributo de la clase `next_page_name` sea `None`. Este método crea una página nueva de nombre `pname` y la define como la siguiente a la página actual. Para comprobar que entiendes el código, intenta usar tu mismo esta función. Te recomendamos continuar con la misma convención para el nombre (`o-1`, `o-2`, `o-3`, `o-4`, ...).

**Importante**: debes eliminar manualmente las páginas que crees tú.

### Lo que tienes que hacer tú

En esta parte de la tarea debes completar la clase `ArtworkTable` con 4 de sus funciones faltantes. Como te diste cuenta, actualmente todas las funciones trabajan con los casilleros (o bloques) de las páginas, así que ahora implementarás las funciones para trabajar con tuplas. 

**Muy importante**: el requisito es que para recorrer y modificar tus páginas **solamente** puedes usar los métodos de la clase que ya existen. Esto significa que no puedes abrir las páginas con funciones y métodos de Python, sino que tienes que pasar por las funciones que hemos implementado para ti. Tampoco deberías tener que usar las funciones del módulo `disk_sim.py` directamente, pero puedes hacerlo. Si consideras necesario hacer funciones auxiliares para evitar repetir código, también puedes hacerlo.

#### Funciones a implementar:

- **(0.5 pts)** `has_next_tuple`: esta función debe retornar `True` si a partir de la posición actual (indicada por `current_block`) existe alguna tupla hacia adelante. Esto incluye la página actual y las que vienen. Esto es similar a lo que hace `has_next_block`, solo que esta última retorna `True` ante un siguiente casillero vacío. En cambio, la función que debes implementar tú solamente se encarga de ver si quedan tuplas.
- **(1 pto)** `get_next_tuple`: esta función debe retornar la siguiente tupla a partir de la posición actual. Esto quiere decir que debe ignorar los casilleros vacíos hasta encontrar una tupla en la página actual o las siguientes. El valor de `current_block` debe quedar *seteado* en el lugar que se encontró la tupla (tanto en términos del casillero y la página).
- **(1 pto)** `insert_into_table`: esta función debe recorrer la tabla desde el principio hasta insertar los valores de `id_artwork` y `name_artwork` en el primer espacio vacío disponible. Además debes asegurarte que aquel `id` no exista en la tabla desde antes. Para esto probablemente necesites recorrer la tabla dos veces. Si en ninguna página hay un casillero vacío debes agregar una página nueva e insertar ahí la tupla.
- **(0.5 pts)** `get_all_elements_from_table`: esta función debe imprimir todas las tuplas de la tabla. Obviamente, no se deben imprimir los `None` de los espacios vacíos.
- **(1 pto)** `get_tuple_by_id`: esta función debe posicionar el valor de `current_block` de forma consistente con la posición donde está la tupla con el `id_artwork` (considerando página y posición en esa página). Además debe retornar la tupla. Si la tupla no existe debe retornar `None`.

Lo que te vamos a revisar es la definición de estos métodos en la clase definida más arriba. Puedes añadir código abajo para mostrar que tus implementaciones funcionan. Además añade un `readme` a tu respuesta para señalar todo lo que pueda facilitar tu corrección.

### Readme para la respuesta de la primera parte

Escribe aquí tu `readme` 😊.

## Parte 2 (2 pts): simulando un Hash Index

Para este momento ya habrás notado que insertar algo sin repetir el `id` o buscar un elemento por el `id` es costoso pues estás recorriendo toda la tabla. Por lo mismo, vas a simular un **Hash Index**. Para esto tienes que generar un diccionario llamado `hash_index` que como `key` tenga los `id_artwork` de la tabla y como `value` tenga el nombre de la página en que está guardada la tupla. Para esto tendrás que recorrer entera la relación.

#### Funciones a implementar:

**(1 pto) Busqueda con índice**: ahora genera una función fuera de la clase cuya firma es:

```Python
def search_with_index(hash_index, artwork_table, id_artwork):
    # ...
```

Que como parámetro recibe el diccionario `hash_index`, una instancia de la clase `ArtworkTable` y un `id_artwork`. Como *output* la función debe retornar la tupla si es que existe. En caso contrario debe retornar `None`. Está función debe ir directamente a la página y **no debe recorrer la tabla entera**. Para esto puedes iniciar una instancia `ArtworkTable` desde el nombre de la página en la que está la tupla (si es que existe) el nombre lo vas a obtener al preguntarselo al diccionario `hash_index`. Como supondrás, ¡Estamos simuando el comportamiento de un **Hash Index Unclustered**.

**(1 pto) Inserción con índice**: ahora genera una función fuera de la clase cuya firma es:

```Python
def insert_with_index(hash_index, artwork_table, id_artwork, name_artwork):
    # ...
```

que inserte la tupla en la tabla y actualice el índice con el nuevo valor. Para comprobar si el `id` ya existe, ahora en vez de recorrer la tabla, solo se lo deberías preguntar al `hash_index`.
La respuesta a esta pregunta la debes escribir en la celda de abajo. Te vamos a revisar la definición de las funciones. Puedes agregar código para mostrar que tu solución es correcta. Además nuevamente te invitamos añadir un `readme` para facilitar tu corrección.

In [ ]:
# Escribe tu solución acá
hash_index = dict()

# Inicializa el diccionario

def search_with_index(hash_index, artwork_table, id_artwork):
    pass

def insert_with_index(hash_index, artwork_table, id_artwork, name_artwork):
    pass

# Puedes añadir aquí código que muestra como funciona tu solución

## Sobre la corrección de esta tarea

En esta tarea vamos a revisar que tus funciones sean correctas. Estas deben comportarse de manera correcta con los datos de prueba que te pasamos, pero también la probaremos con datos que no necesariamente son iguales a estos. En definitiva, no queremos que tu tarea sea una solución para esta instancia en particular, sino que sirvan parra cualquier tabla. Lo que siempre puedes suponer es que la primera página se va a llamar `o-1` y que las páginas serán de 5 casilleros.

### Readme para la respuesta de la segunda parte

Escribe aquí tu `readme` 😊.

## Bonus (1 pto): eliminar tuplas y compactar la tabla

La clase posee una función `set_empty_on_page` que elimina lo que hay en la posición actual señalada por `current_block`. Puedes averiguar tu mismo como funciona este método. Lo que tienes hacer para el bonus es implementar los métodos `del_tuple_by_id` y `compact`. El primero recibe el id de una obra y debe eliminar esa tupla. El segundo método debe compactar las páginas: esto es generar un nuevo conjunto de páginas (con otro nombre si lo deseas) que sea equivalente a la tabla original, pero que no tenga casilleros vacíos entre medio. Esto es para reducir el número de páginas usadas. Para hacer el método compactar, puedes usar funciones del módulo `disk_sim.py`.

In [ ]:
# Añade código que muestre que tu solución es correcta

### Readme para la respuesta del bonus

Escribe aquí tu `readme` 😊.